In [90]:
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
import time
import re

### 파일 불러오기

In [187]:
# pd.read_csv('time1_title_set.txt')
with open("time1_title_set.txt", 'r', encoding='utf8') as f:
    book_list = [i.replace('\',\n','').replace(' \'','').replace('\'','') for i in f]
len(book_list)
book_list = book_list[::-1]

### 리스트 및 변수 생성 및 초기화

In [188]:
book_titles = list()
book_reviews = list() 
reviews_date = list()

book_abstract = list()
book_abstract_title = list()
book_publish_date = list()
just_review_list = list()

num=int(0)
list_num = int(0)

### 크롬 브라우저 생성하기

In [189]:
url='http://www.yes24.com/24/Category/Display/001001026'
driver = webdriver.Chrome()
driver.get(url)
driver.implicitly_wait(3)

- #정말 부득이한 상황에서 크롤링 중단시, num 변수 번호 출력해보기
- #기존 크롤링중이던 7개책(탭)을 돌리려면 num변수 값을 1을 빼고, 위쪽 코드는 실행하지 말고 다시 아래 크롤링 코드만 다시 실행
- #돌리던 7개책 이후의 7개 책을 돌릴려면 num값을 유지한채 다시 아래 크롤링 코드만 실행

- #아래 크롤링 코드 실행시, 일부 책 리뷰의 경우 마지막 페이지를 2번 이상 긁어오는 문제가 있음
- #따라서, 파일 저장과정해서 drop_duplicates로 중복 삭제해야함(현재 파일 저장 코드에 포함되어 있음)

- #크롤링 끝날 시, 에러날 가능성있음. 에러나도 무시하고 파일 저장하시길...

In [ ]:
while True:
    
    num+=int(1)
    #크롤링을 완료한 경우 프로세스 탈출 및 종료(무한 반복문이라서 필요)
#     if len(book_list[(num-1)*7:num*7])==0:
#         print('process finished')
#         break
        
        
    
    #책 검색결과 탭 7개로 출력을 위한 임시 책 7권 리스트 생성
    try:
        just_book_list = book_list[(num-1)*7:num*7]
#         just_book_list = ['쿠션']
        print( )
        print( )
        print(just_book_list)
    #만약 마지막 책 그룹 항목이 7권이 아닐 경우 예외처리
    except IndexError:
        just_book_list = book_list[(num-1)*7:]
    
    #임시 책 7권 리스트 바탕 7개의 검색결과 탭 생성    
    for i in just_book_list:
        tab = driver.window_handles[0]
        driver.switch_to.window(window_name=tab)
        search = driver.find_element_by_css_selector('#query')
        search.clear() #검색창 입력어 지우기
        search.send_keys(i) #새로운 검색어 입력
        click = driver.find_element_by_css_selector('#yesSForm > fieldset > span.schBtn > button')
        click.send_keys(Keys.CONTROL +"\n") #클릭해서 새 탭으로 열리기
    
    
    
    #책 상세페이지 진입을 위한 과정    
    #1단계: 탭전환 및 방해할만한 요소 제거
    while True:
        #탭 전환과정 - 매번 두번째 탭(index 기준1)을 클릭할 수 있도록 조치
        #만약 두번째 탭이 존재하지 않을 경우 임시 책 7권 리스트를 모두 순환한 것으로 인식하고, 위에서 다시 7권 리스트 생성
        try:
            tab = driver.window_handles[1]
            driver.implicitly_wait(3)
            driver.switch_to.window(window_name=tab)
            
            #도중 오류 발생 방지를 위해 yes24 팝업 광고 제거
            try:
                driver.find_element_by_css_selector('#divYes24SCMEvent > div.yPopBot > div.pTxt > label').click()
#                 time.sleep(2) #테스트
            except ElementNotInteractableException:
                pass

            
            
     #2단계: 실제 책 검색한 책이름과 검색결과 항목(책 제목)과 대조 하기
            #인기순(default값) 기준 첫번째 항목 확인 및 대조
            title = driver.find_element_by_xpath(f'/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]/p[1]/a').text
            if title in just_book_list:
                u=1
                
            #아닐 경우, 정밀한 검색을 위해 나열 기준 '정확도'로 변경 및 상위 리스트 10권 제목 확인
            if title not in just_book_list:
                driver.find_element_by_css_selector('#formTest > ul > li:nth-child(2) > a').send_keys(Keys.ENTER)
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[1]/td[2]/p[1]/a').text
                u=1
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[3]/td[2]/p[1]/a').text
                u=3
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[5]/td[2]/p[1]/a').text
                u=5
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[7]/td[2]/p[1]/a').text
                u=7
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[9]/td[2]/p[1]/a').text
                u=9
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[11]/td[2]/p[1]/a').text
                u=11
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[13]/td[2]/p[1]/a').text
                u=13
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[15]/td[2]/p[1]/a').text
                u=15
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[17]/td[2]/p[1]/a').text
                u=17
            if title not in just_book_list:
                title = driver.find_element_by_xpath('/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[19]/td[2]/p[1]/a').text
                u=18
            #책 제목이 존재할 경우 책 상세페이지로 입장
            #만약 동일한 제목이 없을 경우, 3단계 과정 진행시 오류가 나면서 에러처리로 탭이 닫힘(예외처리 참조)
            if title in just_book_list:
                driver.find_element_by_xpath(f'/html/body/div/div[5]/div/div[2]/div[3]/div[2]/table/tbody/tr[{u}]/td[2]/p[1]/a').click()
                #진행 상황 확인할 수 있도록 하기위해 책 이름 및 번째 책 항목인지 출력
                list_num +=1
                print()
                print(f'[{list_num}] {title}')
         
    
    #3단계: 책 초록 긁어오기
            try:
                publish_date = driver.find_element_by_css_selector('#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_date').text
            except NoSuchElementException:
                publish_date = None
                
            try:
                abstract1 = (driver.find_element_by_css_selector('#infoset_introduce > div.infoSetCont_wrap > table')).text
            except NoSuchElementException:
                try:
                    abstract1 = (driver.find_element_by_css_selector('#infoset_introduce > div.infoSetCont_wrap > div.infoWrap_txt')).text
                except NoSuchElementException:
                    pass
            try:
                abstract2 = (driver.find_element_by_css_selector('#infoset_toc > div.infoSetCont_wrap > div.infoWrap_txt')).text
            except NoSuchElementException:
                try:
                    abstract2 = '' #단원목록이 없는 경우에 해당
                except NoSuchElementException:
                    pass
            
            book_publish_date.append(publish_date)
            book_abstract_title.append(title)
            book_abstract.append((abstract1 + abstract2).replace('\n',' ') )
#             print(title, abstract1 + abstract2[0:10])
#             print()
                
    #4단계: 책 상세페이지에서 들어가 크롤링
            time.sleep(2)
            review_num = 0
            try:
                title = driver.find_element_by_xpath('/html/body/div/div[4]/div[2]/div[1]/div/h2')
                title = title.text
                
#                 #진행 상황 확인할 수 있도록 하기위해 책 이름 및 번째 책 항목인지 출력
#                 list_num +=1
#                 print(f'[{list_num}] {title}')
                    
                for k in list(range(2)):
                    for j in list(range(10)):

                        #리뷰 단일 페이지 크롤링
                        #모든 항목(최대 5개 리뷰) 리뷰 더보기 클릭하기
                        for i in list(range(5)):
                            
                            element = driver.find_element_by_xpath('//*[@id="infoset_reviewContentList"]/div['+str(i+2)+']/div[2]/a/div/span')
                            driver.execute_script("arguments[0].click();", element)
                            time.sleep(0.5) #혹시나 인터넷 속도가 감당 못할 경우 시간 늘리기
            
                            
                            
                        #페이지내 각 리뷰별 리뷰,날짜 텍스트 수집
                        
#                         yes = driver.find_element_by_xpath('//*[@id="infoset_reviewContentList"]/div[2]/div[3]/div[2]').text
                        
#                         print()
#                         print()
#                         print(yes[:10])
#                         print(just_review_list)
                        
                        for i in list(range(5)):
                            review_num += 1
                            print(review_num, end=' ')
                            yes = driver.find_element_by_xpath('//*[@id="infoset_reviewContentList"]/div['+ str(i+2) +']/div[3]/div[2]')
                            date = driver.find_element_by_xpath('//*[@id="infoset_reviewContentList"]/div[2]/div[1]/div/em[4]')

    #                                 print(date.text)
#                                         print(yes.text)
    #                                 print()
                            book_titles.append(title)
                            book_reviews.append(yes.text.replace('\n',' ')) 
                            reviews_date.append(date.text)
                            just_review_list.append(yes.text[:10])
                        
#                         print('-------------------------------------페이지 '+ str(k) + str(j+1) + ' 끝---------------------------------------')
                            
                        #다음 리뷰 페이지로 진입
                        yes = driver.find_element_by_css_selector('#infoset_reviewContentList > div.review_sort.sortBot > div.review_sortLft > div > a:nth-child(' + str(j+4) +')')                    
                        yes.click()
                        time.sleep(1)
                driver.close()
            except NoSuchElementException:                
                driver.close()
                pass
            except ElementNotInteractableException:
                driver.close()
                pass
            except ElementClickInterceptedException :
                driver.close()
                pass
        except NoSuchElementException:                
                driver.close()
        except ElementNotInteractableException:
            driver.close()
#         except ElementClickInterceptedException :
#             driver.close()
#             break
        except IndexError:
#             print('start new books process')
            break
        
    
#  #infoset_reviewContentList > div.review_sort.sortBot > div.review_sortLft > div > a:nth-child(10)       



['콰이어트', '코리안 탱크, 최경주', '켄 블랜차드의 리더의 심장', '커뮤니케이션 불변의 법칙', '카이로스', '카네기 인간관계론', '칭찬은 고래도 춤추게 한다']

[96] 콰이어트
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 
[97] 코리안 탱크, 최경주
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 
[98] 켄 블랜차드의 리더의 심장
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 
[99] 커뮤니케이션 불변의 법칙

[100] 카이로스
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 
[101] 카네기 인간관계론
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 
[231] 오케이아웃도어닷컴에 OK는 없다
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 
[232] 오리진이 되라
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 
[233] 오래된 나를 떠나라
1 2 3 4 5 6 7 8 9 10 
[234] 오늘도 나에게 박수를 보낸다
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 

#### 리뷰파일과 초록파일 따로 저장해야함, 크롤링 완료 후 아래 코드 모두 실행하기!

### 리뷰 파일 저장하기
- 최종 파일 이름(time부분) 수정 필수

In [155]:
reviews = pd.DataFrame(book_reviews)
titles = pd.DataFrame(book_titles)
dates = pd.DataFrame(reviews_date)
final_data= pd.concat([titles, reviews, dates],axis=1)
final_data = final_data.drop_duplicates(keep='first')
final_data

,0,0,0
0,1만 시간의 법칙,저희 목사님의 구매 추천을 통하여 1만 시간의 법칙을 구매하게 되었습니다. 1만 시...,2020-03-18
1,1만 시간의 법칙,이 책은 읽어보려고 사둔지 몇 해가 지난 것 같다. 사두고 미뤄뒀던 이유는 이 책의...,2020-03-18
2,1만 시간의 법칙,<1만 시간의 법칙>만큼 제목이 모든 내용을 담고 있는 책도 드물 것이다. 요약하면...,2020-03-18
3,1만 시간의 법칙,"생각 / 하루의 3시간씩 10년을 투자하면 1만시간이 되고, 1만 시간을 투자하면 ...",2020-03-18
4,1만 시간의 법칙,"한 때 이 책의 제목 ""1만시간의 법칙""이 온 인구에 회자 된 적이 있었다. 어떤 ...",2020-03-18
5,1만 시간의 법칙,1만시간의 법칙을 처음 접하게 된건 드라마 <응답하라 1994>에 1만 시간의 법칙...,2014-05-28
6,1만 시간의 법칙,"집 정리를 하다가, 어라? 이런 책이 집에 있었나? 하고 집어든 책. 두껍지도 않고...",2014-05-28
7,1만 시간의 법칙,"모차르트는 신동이라는 이미지가 너무 강해서, 특출난 재능을 타고난 사람이 그 재능을...",2014-05-28
8,1만 시간의 법칙,"우연히 지하철 판매점에서 보고, 살까 망설였던 책이다. 좋은 기회에 잘 샀다라는 생...",2014-05-28
9,1만 시간의 법칙,1만 시간의 법칙! 듣던 것에 비하면 책 내용은 조금 실망이었다. 워낙 자기계발 서...,2014-05-28


In [135]:
final_data.to_csv('time1 탑100 리뷰.csv', encoding='cp949')

### 초록 파일 저장하기
- 최종 파일 이름(time부분) 수정 필수

In [110]:
book_abstract_title = pd.DataFrame(book_abstract_title)
book_abstract = pd.DataFrame(book_abstract)
final_data= pd.concat([book_abstract_title, book_abstract],axis=1)
final_data = final_data.drop_duplicates(keep='first')
final_data

,0,0
0,1% 리더만 아는 유머의 법칙,"지식 중심의 사회에서 게임, 놀이, 재미라는 감성에 빠진 시장을 선도하기 위해서는 ..."
1,1% 행운,감동의 눈물과 웃음으로 용기와 힘을 북돋우는 따뜻한 성공서\n‘영혼을 위한 닭고기수...
2,10 10 10 텐 텐 텐 인생이 달라지는 선택의 법칙,무수한 선택 앞에서 망설이거나 중요한 결정을 앞두고 딜레마에 빠진 모든 현대인들에게...
3,100달러로 세상에 뛰어들어라,"단돈 100달러로 좁은 사무실에서 벗어나\n전 세계를 무대로 펼친,\n당신만의 꿈을..."
4,100세 현역 의사의 스트레스 내려놓기 연습,이 책의 저자 히노하라 시게아키 박사(100). 1911년 10월4일생인 그는 얼마...
5,"10년 후, 나를 디자인한다",2년에 걸친 기획 끝에 과학동아 전문기자 9인이 발로 뛰며 캐낸 진로 설계 비밀. ...
6,10미터만 더 뛰어봐!,한때 20억원이 넘는 빚을 지고 자살을 결심했었으나 단돈 130만원으로 재기에 도전...
7,"14살 세상 끝의 좌절, 23살 세상 속으로의 도전",“꿈이 있다면 가난도 소심함도 아무런 장애가 되지 않는다!”\n왕따에서 유엔 대표까...
8,18분,"열심히 산 결과, 내게 주어진 것은 무엇인가? 성공도 행복도 갖지 못한 채 모래알처..."
9,18시간 몰입의 법칙,이 책에서는 3~4시간의 잠과 18시간 몰입의 법칙을 통해 성공한 한국인들에 대해 ...


In [ ]:
final_data.to_csv('time1 탑100 초록.csv', encoding='cp949')